Загрузка обучающего набора из csv-файла

In [1]:
import csv
import numpy as np
from pandas import read_csv
from keras.utils import np_utils 

def parse_csv():
    X = read_csv('training.csv') 
    A = np.array(X)
    A = A[0:,30]
    res = []
    for i in A:
        res.append([int(x) for x in i.split(' ')])
    return res 
    
x_train = parse_csv()
y_train = [1] * len(x_train)
y_train = np_utils.to_categorical(y_train, 2) 

Using TensorFlow backend.


Загрузка тестового изображения

In [7]:
import csv
import numpy as np
from pandas import read_csv
from keras.utils import np_utils 
from PIL import Image

def read_jpeg(path):
    im = Image.open(path).convert('L')
    X = list(im.getdata())
    X = np.array(X)
    return X
    
x_test = read_jpeg('data/face.jpg')
y_test = []
y_test.append(1)
y_test = np_utils.to_categorical(y_test, 2) 

Загрузка обучающего набора из папки с изображениями

In [15]:
import os
import scipy.io as sio
import scipy

def read_jpeg_training_data():
    x_train = []
    y_train = []
    dirpath = 'data/positive'
    img_ext = '.jpg' 
    img_names = [ os.path.join(dirpath,x) for x in os.listdir( dirpath ) if x.endswith(img_ext) ]
    for i in img_names:
        x_train.append(read_jpeg(i))
        y_train.append(1)  
    dirpath = 'data/negative'
    img_names = [ os.path.join(dirpath,x) for x in os.listdir( dirpath ) if x.endswith(img_ext) ]
    for i in img_names:
        x_train.append(read_jpeg(i))
        y_train.append(0)  
    return x_train, y_train, len(img_names)    

x_train, y_train, set_size = read_jpeg_training_data()
y_train = np_utils.to_categorical(y_train, 2) 

Определение гиперпараметров модели

In [10]:
height, width, deph = 96,96,1
batch_size = 128 
num_epochs = 1 
hidden_size = 512 
num_classes = 2 
conv_depth_1 = 24
pool_size = (2, 2)
kernel_size = 2
drop_prob_1 = 0.25
drop_prob_2 = 0.5

Подготовка обучающего набора

In [11]:
import numpy

x_train = numpy.array(x_train)
x_train = x_train.astype('float32')
x_train /= 225
x_train = x_train.reshape(set_size,1,height,width)

Создание и компиляция модели

In [14]:
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
from keras.utils import np_utils
from keras.models import Model, Sequential  

inp = Input(shape=(1, height, width))
conv_1 = Convolution2D(24, 2, 2, border_mode='same', activation='relu')(inp)
drop_1 = Dropout(drop_prob_1)(conv_1)
flat_1=Flatten()(drop_1)

hidden_1 = Dense(batch_size, activation='relu')(flat_1)
drop_2 = Dropout(drop_prob_2)(hidden_1)
out = Dense(num_classes, activation='softmax')(drop_2)

model = Model(input=inp, output=out)

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])


Обучение модели

In [14]:
model.fit(x_train, y_train, batch_size = batch_size, nb_epoch = num_epochs, verbose=1)

Epoch 1/1
2/2 [==============================] - 0s - loss: 0.6940 - acc: 0.5000


Подготовка тестового изображения

In [19]:
y_test = np.array(y_test)
x_test = numpy.array(x_test)
x_test = x_test.reshape(1,1,height,width)
x_test = x_test.astype('float32')
x_test /= 225

Оценка модели на тестовом наборе

In [20]:
model.evaluate(x_test, y_test, verbose=1) 

1/1 [==============================] - 0s


[0.68372422456741333, 1.0]

Сериализация слоёв и весов

In [1]:
model_json = model.to_json()
with open("network.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("network.csv")

NameError: name 'model' is not defined

Загрузка модели из файлов со слоями и весами

In [4]:
from keras.models import model_from_json

json_file = open('network.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("network.csv")

In [13]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])